In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

sns.set(style='white', context='notebook', palette='deep')

In [ ]:
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.4f}".format(x)})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('./drive/My Drive/DACON/data/train.csv')
train_copy = train.copy()

In [ ]:
# define train set
x_tot = np.divide(train_copy.iloc[:,3:].values,255)
x_tot = x_tot.reshape(-1,28,28,1)
y_tot = train_copy['digit'].values.copy()

# split training and validation set
x_train, x_val, y_train, y_val = train_test_split(x_tot,y_tot,test_size=0.1,random_state=21)

In [ ]:
x_train_arr = []
y_train_arr = []

for num in range(10):
    # reshape and normalization
    x_train_arr.append(x_train)
    # label encoding
    y_label = y_train.copy()
    y_label[y_label != num]= -1
    y_label[y_label == num]= 1
    y_label[y_label == -1] = 0
    y_label = to_categorical(y_label,num_classes = 2)
    y_train_arr.append(y_label)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "sigmoid"))


In [ ]:
"""
# set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

# with data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, 
                              validation_data = (x_val,y_val),
                              steps_per_epoch=x_train.shape[0]// batch_size
                              , callbacks=[learning_rate_reduction])
"""

In [ ]:
model_arr = []
val_acc = []

epochs = 20
batch_size = 50

for num in range(10):
    binary = clone_model(model)

    optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    binary.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])

    # split training and validation set
    x_train_d, x_val_d, y_train_d, y_val_d = train_test_split(x_train_arr[num],y_train_arr[num],test_size=0.1,random_state=15)

    history = binary.fit(x_train_d, y_train_d, batch_size=batch_size, epochs = epochs,  validation_data = (x_val_d,y_val_d))

    model_arr.append(binary)
    val_acc.append(history.history['val_accuracy'][19])

    print('\033[31m' + '{} = Val Acc : {}'.format( num, val_acc[num] ) + '\033[0m')

    """# plot the loss and accuracy curves for training and validation 
    fig, ax = plt.subplots(2,1)
    ax[0].plot(history.history['loss'], color='b', label="Training loss")
    ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
    legend = ax[0].legend(loc='best', shadow=True)

    ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
    ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
    legend = ax[1].legend(loc='best', shadow=True)"""

print()
print('digit       val_acc')
for num in range(10):
    print('  {}    {}'.format( num, val_acc[num] ))

In [ ]:
def digit_prob(input_img):
    
    output_label = np.zeros((10,))
    input = input_img.reshape(1,28,28,1)

    for num in range(10):
        label = model_arr[num].predict(input)[0][1]
        output_label[num] = label 

    return output_label

In [ ]:
def classifier(input_imgs):
    
    output_labels = np.zeros((input_imgs.shape[0],))
    label = np.zeros((10,))
    for i in range(output_labels.shape[0]):
        input = input_imgs[i].reshape(1,28,28,1)

        for num in range(10):
            label[num] = model_arr[num].predict(input)[0][1]
    
        output_labels[i] = np.argmax( label )

        if i%100==0 : print(i, end=' ')
    print()

    return output_labels

In [ ]:
true_digit = y_val
prediction = classifier(x_val).astype(np.int)
acc = len(true_digit[true_digit==prediction])/len(true_digit)

print('Accuracy  : {}'.format( acc ))

In [ ]:
error = []
for num in range(10):
    id = np.where(true_digit==num)
    pred = prediction[id]
    error.append( pred[pred!=num] )
    print('{} error rate : {}'.format(num, format(len(error[num])/len(pred),'.4f') ), end='  ')
    print('wrong predictions : {}'.format(error[num]))
print()
cnt = []
error_arr = np.concatenate(error)
for i in range(10):
    cnt.append( len(error_arr[error_arr==i]) )
print('error distribution : {}'.format(cnt))
print('Most error : {}'.format( np.argmax(cnt)))

In [ ]:
def find_error(num):
    
    x = x_val[ np.where(true_digit==num) ]
    for i in range(len(x)):
        out = digit_prob(x[i])
        loc = np.argmax(out)
        print('Predicted {} : {}'.format(loc,format(out[loc],'.4f')),end='   ')
        print('True {} : {}'.format(num,format(out[num],'.4f')))
        if (out[loc]>0.5)&(loc != num) : 
            plt.figure(figsize=(5,5))
            plt.imshow(x[i].reshape(28,28),cmap='gray')
            plt.title('Predicted {} : {}'.format(loc,format(out[loc],'.4f')), color='w', fontsize=15)
            plt.axis('off')

In [ ]:
find_error(2)